In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
ORSkey=os.environ['ORSkey']

: 

In [ ]:
import numpy as np
import pandas as pd
import folium
import ast
import openrouteservice as ors
import random

: 

In [ ]:
locations = pd.read_csv("../foodstuffs-data/FoodstuffsLocations.csv")
locations.head()

: 

In [ ]:
selected_saturday = pd.read_csv("../linear-program/output/SelectedRoutesSaturday.csv")
selected_saturday.head()

: 

In [ ]:
selected_weekday = pd.read_csv("../linear-program/output/SelectedRoutesWeekday.csv")
selected_weekday.head()

: 

In [ ]:
def initialise_map(locations):
    coords = locations[['Long', 'Lat']] # Mapping packages work with Long, Lat arrays
    coords = coords.to_numpy().tolist() # Make the arrays into a list of lists.
    # Folium, however, requires Lat, Long arrays - so a reversal is needed.
    # coords[0] is the warehouse
    m = folium.Map(location = list(reversed(coords[2])), zoom_start=10.25)
    for i in range(len(locations)):
        if locations.Type[i] == "Four Square":
            icon_url = "store_green.png"
        elif locations.Type[i] == "New World":
            icon_url = "store_red.png"
        elif locations.Type[i] == "Pak 'n Save":
            icon_url = "store_orange.png"
        elif locations.Type[i] == "Warehouse":
            icon_url = "warehouse.png"
        icon = folium.features.CustomIcon(icon_url, icon_size=(16, 16))
        folium.Marker(
            list(reversed(coords[i])),
            popup = locations.Supermarket[i],
            icon = icon # folium.Icon(color = iconCol)
        ).add_to(m)
    return m

: 

In [ ]:
def find_driving_route(coords):
    client = ors.Client(key=ORSkey)
    
    return client.directions(
        coordinates = coords,
        profile = 'driving-hgv', # can be driving-car, driving-hgv, etc.
        format='geojson',
        validate = False
    )

: 

In [ ]:
def plot_routes(locations, routes, m):
    route_list = [ast.literal_eval(route) for route in routes.Route.values.tolist()]
    for route in route_list:
        coord_list = []
        for node in route:
            coord = locations[locations.Supermarket==node][['Long', 'Lat']]
            coord = coord.to_numpy().tolist()
            coord_list.append(coord[0])
        
        driving_route = find_driving_route(coord_list)
        coordinates = driving_route['features'][0]['geometry']['coordinates']
        folium.PolyLine(
            locations = [list(reversed(coord)) for coord in coordinates],
            color = '#' + ''.join([random.choice('ABCDEF0123456789') for i in range(6)])
        ).add_to(m)
    return m

: 

In [ ]:
initialise_map(locations)

: 

In [ ]:
saturday_map = initialise_map(locations)
plot_routes(locations, selected_saturday, saturday_map)

: 

In [ ]:
weekday_map = initialise_map(locations)
plot_routes(locations, selected_weekday, weekday_map)

: 